In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
# file path - this for linux windows you will need "//"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

Wednesday_data = "path"

# Read the new data from the second CSV file
df = pd.read_csv(Wednesday_data, sep=',')


In [3]:
df.shape

(692703, 79)

In [4]:
#df.describe()
df.isnull().any().any()

True

In [5]:
#PREPROCCSING
# Replace infinite or very large values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

#drop any instance that is missing a feture
df = df.dropna()
df.isnull().any().any() # check if we have nan values.

False

In [6]:
#df.head(10)
df[' Label'].unique()

array(['BENIGN', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk',
       'DoS GoldenEye', 'Heartbleed'], dtype=object)

In [7]:
df.shape

(691406, 79)

In [8]:
features = [' Fwd IAT Min',
    'Init_Win_bytes_forward',
    ' Destination Port',
    ' Bwd Packet Length Min',
    ' Init_Win_bytes_backward',
    ' Subflow Fwd Bytes',
    ' Total Fwd Packets',
    ' Total Length of Bwd Packets',
    ' Bwd Packet Length Mean',
    ' Fwd Packet Length Min',
    ' PSH Flag Count',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Packet Length Variance',
    'Active Mean']
x = df[features]

y = df[' Label']

In [9]:
x

,Fwd IAT Min,Init_Win_bytes_forward,Destination Port,Bwd Packet Length Min,Init_Win_bytes_backward,Subflow Fwd Bytes,Total Fwd Packets,Total Length of Bwd Packets,Bwd Packet Length Mean,Fwd Packet Length Min,PSH Flag Count,Flow Packets/s,Flow IAT Mean,Packet Length Variance,Active Mean
0,0,255,80,6,946,6,1,6,6.000000,6,0,52.208416,38308.000000,0.000000,0.0
1,1,29200,389,0,260,172,11,326,65.200000,0,1,33402.922760,31.933333,3195.595588,0.0
2,1,29200,88,0,2081,3150,10,3150,525.000000,0,1,14611.872150,73.000000,451250.132400,0.0
3,2,29200,389,0,0,3452,17,6660,555.000000,0,1,1907.141918,543.071429,496537.374700,0.0
4,1,29200,88,0,2081,3150,9,3152,525.333333,0,1,13736.263740,78.000000,496440.116700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,4,-1,53,76,-1,112,4,152,76.000000,28,0,186.248642,6443.000000,548.571429,0.0
692699,2,-1,53,181,-1,84,2,362,181.000000,42,0,12345.679010,108.000000,5796.300000,0.0
692700,4,1006,58030,6,0,31,2,6,6.000000,0,0,36585.365850,41.000000,267.333333,0.0
692701,1,-1,53,128,-1,192,6,256,128.000000,32,0,7.628965,149805.000000,1792.000000,0.0


In [10]:
y

0         BENIGN
1         BENIGN
2         BENIGN
3         BENIGN
4         BENIGN
           ...  
692698    BENIGN
692699    BENIGN
692700    BENIGN
692701    BENIGN
692702    BENIGN
Name:  Label, Length: 691406, dtype: object

In [11]:
# First score - using rf with 15 features from another article

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#standardScaler normalization
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

rf = RandomForestClassifier(max_depth=10, random_state=45)
rf.fit(X_train_transformed, y_train)
y_predicted = rf.predict(X_test_transformed)

accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted, average = 'macro')
recall = recall_score(y_test, y_predicted, average = 'macro')
f1 = f1_score(y_test, y_predicted, average = 'macro')
conf_matrix = confusion_matrix(y_test, y_predicted)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9989514181166023
Precision: 0.9978040712901887
Recall: 0.9947862778845332
F1 Score: 0.9962864557648762
Confusion Matrix:
 [[87818     0    87     3     1     0]
 [   21  2001     4     2     0     0]
 [    8     0 46077     0     0     0]
 [    4     0     0  1084     4     0]
 [    9     0     0     2  1155     0]
 [    0     0     0     0     0     2]]


In [13]:
# Second score - using XGB with 15 features from another article

In [14]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# create train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Fit and transform the labels in both training and test sets
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

#standardScaler normalization
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Initialize XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_transformed, y_train_encoded)
y_predicted_encoded = xgb_model.predict(X_test_transformed)

# Inverse transform the encoded labels to get the original class labels
y_predicted = label_encoder.inverse_transform(y_predicted_encoded)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted, average = 'macro')
recall = recall_score(y_test, y_predicted, average = 'macro')
f1 = f1_score(y_test, y_predicted, average = 'macro')
conf_matrix = confusion_matrix(y_test, y_predicted)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9997107360321662
Precision: 0.9968704370641751
Recall: 0.9968580210459592
F1 Score: 0.9968638764657592
Confusion Matrix:
 [[87989     0    14     2     1]
 [    0  2019     4     2     0]
 [    0     2 45999     0     1]
 [    2     0     0  1095     6]
 [    2     0     0     4  1140]]


In [17]:
# Third score - Using ada boost with 15 features from another article

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# StandardScaler normalization
scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_val_transformed = scaler.transform(X_val)
X_test_transformed = scaler.transform(X_test)

# Initialize and train the AdaBoost classifier
ada_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)
ada_classifier.fit(X_train_transformed, y_train)

# Predictions on validation set
y_val_predicted = ada_classifier.predict(X_val_transformed)

# Predictions on test set
y_test_predicted = ada_classifier.predict(X_test_transformed)

# Calculate evaluation metrics for validation set
accuracy_val = accuracy_score(y_val, y_val_predicted)
precision_val = precision_score(y_val, y_val_predicted, average = 'macro')
recall_val = recall_score(y_val, y_val_predicted, average = 'macro')
f1_val = f1_score(y_val, y_val_predicted, average = 'macro')
conf_matrix_val = confusion_matrix(y_val, y_val_predicted)

# Print evaluation metrics for validation set
print("Validation Set Results:")
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("Confusion Matrix:\n", conf_matrix_val)

# Calculate evaluation metrics for test set
accuracy_test = accuracy_score(y_test, y_test_predicted)
precision_test = precision_score(y_test, y_test_predicted, average = 'macro')
recall_test = recall_score(y_test, y_test_predicted, average = 'macro')
f1_test = f1_score(y_test, y_test_predicted, average = 'macro')
conf_matrix_test = confusion_matrix(y_test, y_test_predicted)

# Print evaluation metrics for test set
print("\nTest Set Results:")
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("Confusion Matrix:\n", conf_matrix_test)


Validation Set Results:
Accuracy: 0.7553549992045241
Precision: 0.7553549992045241
Recall: 0.7553549992045241
F1 Score: 0.7553549992045241
Confusion Matrix:
 [[40197  1247   955  1414    16     0]
 [   11   738    37   248     0     0]
 [  142 11944 11056     0     0     0]
 [   24   305     0   164    66     0]
 [   17   167     0   322    70     0]
 [    0     0     0     0     0     1]]

Test Set Results:
Accuracy: 0.7531999826441619
Precision: 0.7531999826441619
Recall: 0.7531999826441619
F1 Score: 0.7531999826441619
Confusion Matrix:
 [[40294  1271   970  1493    26     0]
 [   19   753    39   262     0     0]
 [  109 11969 10822     0     0     0]
 [   15   322     0   140    60     0]
 [   18   163     1   327    67     0]
 [    0     0     0     0     0     1]]
